In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from aleister_producer  import AleisterFeedAgent

In [4]:
afa = AleisterFeedAgent()

In [5]:
afa.setup_data_provider()

In [ ]:
afa.subscribe_realtime_data()

In [ ]:
afa.stop_subscribe_realtime_data()

In [11]:
afa.init_db()

In [29]:
data = afa.fetch_hist_realtime_data(20211201,20211211, ['orderbook'])[20211209]['orderbook']

In [5]:
afa.db_accesser.dao.find_one("orderbook")

AttributeError: 'NoneType' object has no attribute 'dao'

In [10]:
afa.start_realtime_fetch()

In [7]:
#!/usr/bin/env python
import pika

connection = pika.BlockingConnection(
    pika.ConnectionParameters(host='localhost'))

channel = connection.channel()

channel.queue_declare(queue='aleister')

def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n - 1) + fib(n - 2)

def on_request(ch, method, props, body):
    self.logger.info(f"[DONE] via MQ. Host={method}, pro:{props}, cd:{ch}")
    n = int(body)

    print(" [.] fib(%s)" % n)
    response = fib(n)

    ch.basic_publish(exchange='',
                     routing_key=props.reply_to,
                     properties=pika.BasicProperties(correlation_id = \
                                                         props.correlation_id),
                     body=str(response))
    ch.basic_ack(delivery_tag=method.delivery_tag)

channel.basic_qos(prefetch_count=1)
channel.basic_consume(queue='aleister', on_message_callback=on_request)

print(" [x] Awaiting RPC requests")
channel.start_consuming()

 [x] Awaiting RPC requests


NameError: name 'self' is not defined

In [8]:
import pika,time
class MessageWorker(object):
    def __init__(self):
        self.connection = pika.BlockingConnection(
        pika.ConnectionParameters(host='localhost'))
        self.channel = self.connection.channel()
        self.channel.queue_declare(queue='aleister')

    def on_request(self, ch, method, props, body):
        print('MessageWorker get message ------', body)
        response = self.task(body)
        print('MessageWorker callback on queue ------', props.reply_to)
        ch.basic_publish(exchange='',
                        routing_key=None,
                        properties=pika.BasicProperties(
                            correlation_id=props.correlation_id),
                        body=str(response))
        print('MessageWorker send message to MessageSender')
        ch.basic_ask(delivery_tag=method.delivery_tag)

    def work(self):
        self.channel.basic_qos(prefetch_count=2)
        self.channel.basic_consume(on_message_callback=self.on_request, queue='aleister')
        print('[x]Waiting message...')
        self.channel.start_consuming()


    def task(self,body):
        time.sleep(3)
        return body



worker = MessageWorker()
worker.work()

[x]Waiting message...
MessageWorker get message ------ b'{"orderbook": [{"time": "20211209064344046000", "symbol": "BTC", "bids0": 5483120.0, "bids0_size": 0.0125, "bids1": 5483072.0, "bids1_size": 0.088, "bids2": 5483050.0, "bids2_size": 0.04, "bids3": 5483040.0, "bids3_size": 0.1625, "bids4": 5482961.0, "bids4_size": 0.0291, "asks0": 5486394.0, "asks0_size": 0.0003, "asks1": 5486750.0, "asks1_size": 2.5, "asks2": 5487200.0, "asks2_size": 0.2, "asks3": 5487350.0, "asks3_size": 0.4, "asks4": 5487400.0, "asks4_size": 0.5}, {"time": "20211209064344552000", "symbol": "BTC", "bids0": 5483200.0, "bids0_size": 0.0125, "bids1": 5483120.0, "bids1_size": 0.0125, "bids2": 5483078.0, "bids2_size": 0.088, "bids3": 5483050.0, "bids3_size": 0.04, "bids4": 5483041.0, "bids4_size": 0.0191, "asks0": 5486394.0, "asks0_size": 0.0003, "asks1": 5486900.0, "asks1_size": 2.5, "asks2": 5487000.0, "asks2_size": 0.1, "asks3": 5487150.0, "asks3_size": 0.1, "asks4": 5487200.0, "asks4_size": 0.5}, {"time": "202112

AssertionError: A non-string value was supplied for self.routing_key